# Production Voronoi-Based Pu-Cl Cluster Analysis

This notebook implements the complete analysis pipeline for Pu-Cl cluster systems using Voronoi tessellation from OVITO.

## Overview

This analysis pipeline includes:
1. Voronoi tessellation using OVITO
2. Weighted coordination analysis
3. Shared-anion network construction
4. Oligomer characterization
5. Temporal stability analysis

See `spec.md` for detailed specification.


In [1]:
# All imports
import glob
import numpy as np

# OVITO imports
from ovito.io import import_file

# Local utility imports
from utils import (
    get_pipeline_info,
    extract_particle_properties,
    # Phase 2: Voronoi Tessellation
    apply_voronoi_analysis,
    extract_voronoi_bonds,
    extract_voronoi_particle_properties,
    compute_voronoi_face_areas,
    perform_voronoi_tessellation,
)

# Plotting imports
from plots import setup_plot_style


In [2]:
# Configure your input structure/trajectory file
input_path = '/pscratch/sd/p/pvashi/irp/irp_mace_l_2/irp/density/NaCl-PuCl3/x0.27/T1000K/dump.lammpstrj' # TODO: Set path, e.g., '/pscratch/sd/p/pvashi/irp/irp_mace_l_2/irp/density/NaCl-PuCl3/x0.67/T900K/dump.lammpstrj'

# Analysis parameters (will be used in later phases)
MIN_AREA_PERCENT = 0.0  # Minimum area threshold as percent of total
USE_RADII = False  # Whether to use atomic radii for Voronoi tessellation

print("Configuration loaded")

Configuration loaded


## Data Loading

Load trajectory file and extract basic pipeline information.


In [3]:
# Load trajectory file using OVITO
if input_path is not None:
    # Load with OVITO pipeline
    pipeline = import_file(input_path, multiple_frames=True)
    print(f"Loaded trajectory with {pipeline.source.num_frames} frames")
    
    # Choose frame index (use last frame by default)
    frame = pipeline.source.num_frames - 1
    
    # Get basic pipeline information using utils
    info = get_pipeline_info(pipeline)
    print("\nPipeline Information:")
    print(f"  Number of frames: {info['num_frames']}")
    print(f"  Number of particles: {info['num_particles']}")
    print(f"  Species: {info['species']}")
    
    # Display cell information from utils
    if info['cell_info']:
        cell_info = info['cell_info']
        print("\nCell Information:")
        if cell_info.get('matrix') is not None:
            print("  Cell matrix:")
            print(np.array(cell_info['matrix']))
        print(f"  PBC: {cell_info.get('pbc', 'N/A')}")
    
    # Set up standardized plotting style
    setup_plot_style()
    print("\nStandardized plotting style configured")
else:
    print("input_path not set. Please configure the trajectory file path above.")
    pipeline = None
    frame = None


Loaded trajectory with 14171 frames

Pipeline Information:
  Number of frames: 14171
  Number of particles: 3584
  Species: ['Pu', 'Cl', 'Na']

Cell Information:
  Cell matrix:
[[47.94052381  0.          0.          0.05055929]
 [ 0.         47.94052381  0.          0.05055929]
 [ 0.          0.         47.94052381  0.05055929]]
  PBC: (True, True, True)

Standardized plotting style configured


## Basic Pipeline Info Extraction

Extract and display particle properties from a specific frame.


In [4]:
# Extract data from chosen frame
if pipeline is not None and frame is not None:
    frame_data = pipeline.compute(frame)
    properties = extract_particle_properties(frame_data)
    
    print(f"Frame {frame} Properties:")
    print(f"  Positions shape: {properties['positions'].shape if properties['positions'] is not None else 'N/A'}")
    print(f"  Species shape: {properties['species'].shape if properties['species'] is not None else 'N/A'}")
    if properties['species'] is not None:
        unique_species = np.unique(properties['species'])
        print(f"  Unique species in frame: {unique_species}")
    print(f"  Cell matrix shape: {properties['cell_matrix'].shape if properties['cell_matrix'] is not None else 'N/A'}")
    print(f"  Periodic boundary conditions: {properties['pbc']}")
else:
    print("Pipeline not loaded. Please load a trajectory file first.")

Frame 14170 Properties:
  Positions shape: (3584, 3)
  Species shape: (3584,)
  Unique species in frame: ['Cl' 'Na' 'Pu']
  Cell matrix shape: (3, 4)
  Periodic boundary conditions: (True, True, True)


## Phase 2: Voronoi Tessellation

Apply VoronoiAnalysisModifier with generate_bonds=True and extract results.


In [5]:
# Perform Voronoi tessellation

voronoi_results = perform_voronoi_tessellation(
    pipeline=pipeline,
    frame=frame,
    use_radii=USE_RADII,
    edge_threshold=0.0,
    compute_surface_mesh=False,
)

# Extract results
data = voronoi_results['data']
bonds = voronoi_results['bonds']
particle_props = voronoi_results['particle_properties']
face_areas = voronoi_results['face_areas']


In [6]:
print(face_areas)

[4.84987287 0.18711997 9.38557883 ... 0.74120721 4.42113363 6.50210591]


In [8]:
print('face_areas',len(face_areas),'bonds',len(bonds['pairs']))

face_areas 27458 bonds 27458
